In [ ]:
from PIL import Image

import torch
import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms, models

from utils import FCNResNet101
from visualize import show


image_path = "./skin_tone_val/002.jpg"
ckpt_path = "fcn_resnet101_skin_920a9fbd.pth"
score_threshold = 0.5
skin_tone = 0.1
mean = torch.tensor((0.485, 0.456, 0.406)).reshape(3, 1, 1)
std = torch.tensor((0.229, 0.224, 0.225)).reshape(3, 1, 1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ckpt = torch.load(ckpt_path, map_location=device)
model = FCNResNet101(score_threshold)
model.load_state_dict(ckpt)
model.to(device)
model.eval()

del ckpt

image = Image.open(image_path).convert("RGB")
image = transforms.ToTensor()(image)
img = (image - mean) / std

with torch.no_grad():
    img = img.unsqueeze(0).to(device)
    res = model(img)

target = [{"masks": res.squeeze(0).mul(skin_tone)}]
show(image, target)

print("Done!")